In [1]:
# Import dependencies
import sqlite3
import pandas as pd
import numpy as np
from numpy import nan

In [2]:
# Connect to database
conn = sqlite3.connect('database.sqlite')

In [4]:
# Read sqlite query results into a pandas DataFrame
scholl_df = pd.read_sql_query('SELECT * from schools_table',conn)
scholl_df.head()

,index,school_id,school_name,type,size,budget
0,0,0,Stewart High School,Charter,1208,740504
1,1,1,Long High School,Charter,628,368636
2,2,2,Hood High School,Charter,930,578460
3,3,3,Lopez High School,District,3428,2248768
4,4,4,Vargas High School,Charter,2479,1574165


In [5]:
# Read sqlite query results into a pandas DataFrame
student_df = pd.read_sql_query('SELECT * from students_table',conn)
student_df.head()

,index,student_id,student_name,gender,grade,school_name,reading_score,math_score
0,0,0,Kevin Logan,M,9th,Stewart High School,93,98
1,1,1,Earl Bush,M,10th,Stewart High School,99,92
2,2,2,Jeffrey Williamson,M,10th,Stewart High School,98,98
3,3,3,Karen Dean,F,10th,Stewart High School,97,77
4,4,4,Rebecca Reyes,F,11th,Stewart High School,89,87


# District Summary

In [ ]:
# Read sqlite query results into a pandas DataFrame

#  School Summary

In [ ]:
# Read sqlite query results into a pandas DataFrame

# Top Performing Schools (By Passing Rate)

In [ ]:
# Read sqlite query results into a pandas DataFrame

# Bottom Performing Schools (By Passing Rate)

# Math Scores by Grade

# Reading Score by Grade

# Scores by School Spending

# Scores by School Size

# Scores by School Type